#Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

##Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

##Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

###urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [1]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [2]:
print soup.prettify()[0:1000]

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml">
 <head>
  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>
  <meta content="#CC0000" name="theme-color"/>
  <script type="text/javascript">
   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {
        window.location = 'http://a.m.espn.go.com/mobilecache/general/apps/sc';
    }
  </script>
  <title>
   2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN
  </title>
  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>
  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>
  <meta content="noodp" name="googlebot"/>
  <meta content="index, follow" na

##Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

###find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [3]:
table_div = soup.find(id='my-players-table')
#print table_div.prettify()[0:4000]

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [4]:
table = table_div.find("table")

###Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [5]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [6]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [7]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


###Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [8]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [9]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

[{u'FGM-FGA': u'11.6-20.2', u'MPG': u'32.0', u'FTM-FTA': u'7.0-7.4', u'FG%': u'.574', u'GP': u'5', u'PLAYER': u'Stephen Curry', u'FT%': u'.946', u'TEAM': u'GS', u'3PM-3PA': u'5.6-10.8', 'POSITION': u'PG', u'PTS': u'35.8', u'3P%': u'.519'}, {u'FGM-FGA': u'10.2-21.0', u'MPG': u'38.2', u'FTM-FTA': u'6.7-7.5', u'FG%': u'.484', u'GP': u'6', u'PLAYER': u'Kevin Durant', u'FT%': u'.889', u'TEAM': u'OKC', u'3PM-3PA': u'2.8-6.7', 'POSITION': u'SF', u'PTS': u'29.8', u'3P%': u'.425'}, {u'FGM-FGA': u'11.2-19.0', u'MPG': u'34.6', u'FTM-FTA': u'5.8-7.6', u'FG%': u'.589', u'GP': u'5', u'PLAYER': u'Blake Griffin', u'FT%': u'.763', u'TEAM': u'LAC', u'3PM-3PA': u'0.0-0.4', 'POSITION': u'PF', u'PTS': u'28.2', u'3P%': u'.000'}, {u'FGM-FGA': u'10.0-21.3', u'MPG': u'36.7', u'FTM-FTA': u'3.5-4.0', u'FG%': u'.469', u'GP': u'6', u'PLAYER': u'Damian Lillard', u'FT%': u'.875', u'TEAM': u'POR', u'3PM-3PA': u'3.8-9.0', 'POSITION': u'PG', u'PTS': u'27.3', u'3P%': u'.426'}, {u'FGM-FGA': u'10.2-21.2', u'MPG': u'36.0',

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

##Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [10]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

3P%   3PM-3PA   FG%    FGM-FGA   FT%  FTM-FTA GP   MPG             PLAYER  \
0  .519  5.6-10.8  .574  11.6-20.2  .946  7.0-7.4  5  32.0      Stephen Curry   
1  .425   2.8-6.7  .484  10.2-21.0  .889  6.7-7.5  6  38.2       Kevin Durant   
2  .000   0.0-0.4  .589  11.2-19.0  .763  5.8-7.6  5  34.6      Blake Griffin   
3  .426   3.8-9.0  .469  10.0-21.3  .875  3.5-4.0  6  36.7     Damian Lillard   
4  .344   1.8-5.3  .480  10.2-21.2  .857  5.0-5.8  6  36.0  Russell Westbrook   

  POSITION   PTS TEAM  
0       PG  35.8   GS  
1       SF  29.8  OKC  
2       PF  28.2  LAC  
3       PG  27.3  POR  
4       PG  27.2  OKC

Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [11]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

PLAYER POSITION TEAM GP   MPG   PTS    FGM-FGA   FG%   3PM-3PA  \
0      Stephen Curry       PG   GS  5  32.0  35.8  11.6-20.2  .574  5.6-10.8   
1       Kevin Durant       SF  OKC  6  38.2  29.8  10.2-21.0  .484   2.8-6.7   
2      Blake Griffin       PF  LAC  5  34.6  28.2  11.2-19.0  .589   0.0-0.4   
3     Damian Lillard       PG  POR  6  36.7  27.3  10.0-21.3  .469   3.8-9.0   
4  Russell Westbrook       PG  OKC  6  36.0  27.2  10.2-21.2  .480   1.8-5.3   

    3P%  FTM-FTA   FT%  
0  .519  7.0-7.4  .946  
1  .425  6.7-7.5  .889  
2  .000  5.8-7.6  .763  
3  .426  3.5-4.0  .875  
4  .344  5.0-5.8  .857

##Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [12]:
import pandas as pd
import numpy as np
import urllib2
from bs4 import BeautifulSoup


def get_cols(table):
    table_head = table.find(attrs={"class":'colhead'})
    header_cols = table_head.findAll('td')
    
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
           
    return cols

In [13]:
#Take page URL and return the my-players-table 
def open_page_find_table(page):
    page = urllib2.urlopen(page).read()
    soup = BeautifulSoup(page)
    table_div = soup.find(id='my-players-table')
    table = table_div.find("table")
    return table

In [14]:
def get_data(table):
    #Initialize the arrays and find all rows in the table
    table_rows = table.findAll('tr')
    players_stats_array = []
    
    #loop through the rows we found, Ignore the Column Header row
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
      
        #Intialize the arrays for the players stats
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
    
        #Set the player column to the current row
        player_col = row_cols[1]
        #Element <a> contains the payer name, insdie the first <td> element
        player_name = player_col.find('a').string 
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
    
        #Loop through the rest of the Columns and append the values to player_stats
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        
        #Add that player's stats to the player_stats_array   
        players_stats_array.append(player_stats)
            
    return players_stats_array

In [15]:
page_URL = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false'

table = open_page_find_table(page_URL)
cols = get_cols(table)
players_stats_array = get_data(table)

# we need to add to the stats_array and dict instead of reassigning them 
for i in range(1,5):
    x = (i * 40) + 1
    page_URL = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/' + str(x)
    
    table = open_page_find_table(page_URL)
    players_stats_array.extend( get_data(table) )
    
#Now use NumPy array to create a Dataframe
np_array = np.array(players_stats_array)
points_df = pd.DataFrame(np_array, columns=cols)

In [16]:
# Now we need to do the same thing for Assists    
page_URL = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false'

table = open_page_find_table(page_URL)
cols = get_cols(table)
players_stats_array = get_data(table)

# we need to add to the stats_array and dict instead of reassigning them 
for i in range(1,5):
    x = (i * 40) + 1
    page_URL = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/' + str(x)
    
    table = open_page_find_table(page_URL)
    players_stats_array.extend( get_data(table) )

    
#Now use NumPy array to create a Dataframe
np_arr = np.array(players_stats_array)
assists_df = pd.DataFrame(np_arr, columns=cols) 

In [18]:

#Merge dataframes on player
full_df = pd.merge(points_df, assists_df, on = ['PLAYER','POSITION','TEAM','GP','MPG'], how='outer')
full_df.to_csv('HW2.csv')